In [11]:
import numpy as np
import pandas as pd
import json
import gzip
# 'C:\\Users\\emovi\\Desktop\\VibeCaster\\VibeCaster\\data\\Software_5.json'
#C:\Users\emovi\Desktop\VibeCaster\VibeCaster\data\Software_5.json(1).gz

In [13]:
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('C:\\Users\\emovi\\Desktop\\VibeCaster\\VibeCaster\\data\\Industrial_and_Scientific_5.json.gz')

In [17]:
 # summary statistics
print(df.shape) # 
print(df.info())
print(df.describe())

(77071, 12)
<class 'pandas.core.frame.DataFrame'>
Index: 77071 entries, 0 to 77070
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   overall         77071 non-null  float64
 1   verified        77071 non-null  bool   
 2   reviewTime      77071 non-null  object 
 3   reviewerID      77071 non-null  object 
 4   asin            77071 non-null  object 
 5   style           36037 non-null  object 
 6   reviewerName    77044 non-null  object 
 7   reviewText      77060 non-null  object 
 8   summary         77061 non-null  object 
 9   unixReviewTime  77071 non-null  int64  
 10  vote            9620 non-null   object 
 11  image           1719 non-null   object 
dtypes: bool(1), float64(1), int64(1), object(9)
memory usage: 7.1+ MB
None
            overall  unixReviewTime
count  77071.000000    7.707100e+04
mean       4.524062    1.454857e+09
std        0.949668    4.559407e+07
min        1.000000    1.051402e+0

In [18]:
print(df.isna().sum())

overall               0
verified              0
reviewTime            0
reviewerID            0
asin                  0
style             41034
reviewerName         27
reviewText           11
summary              10
unixReviewTime        0
vote              67451
image             75352
dtype: int64


In [19]:
# delete entrys with missin reviewText: 
df.dropna(subset=['reviewText'], inplace=True)

In [20]:
# only keep overall and reviewText
all_columns = df.columns.tolist()

columns_to_keep = ['overall', 'reviewText']

columns_to_drop = [col for col in all_columns if col not in columns_to_keep]

df.drop(columns=columns_to_drop, inplace=True)

In [21]:
print(df)

       overall                                         reviewText
0          5.0  This worked really well for what I used it for...
1          5.0                   Fast cutting and good adheasive.
2          5.0  Worked great for my lapping bench.  I would li...
3          4.0                                      As advertised
4          5.0  seems like a pretty good value as opposed to b...
...        ...                                                ...
77066      5.0  So far it has worked like a champ. Great solut...
77067      5.0  Great quality solid state relay. I used this s...
77068      5.0  Came with everything needed to install in my M...
77069      5.0  Installed a month ago in my Monoprice Maker Se...
77070      5.0  This releaves the mothere board of the current...

[77060 rows x 2 columns]


In [24]:
print(df.describe())

            overall
count  77060.000000
mean       4.524046
std        0.949636
min        1.000000
25%        4.000000
50%        5.000000
75%        5.000000
max        5.000000


In [25]:
# create new col sentiment to train the model on

def classify_sentiment(overall_score):
    if 1 <= overall_score <= 2:
        return 1  # Negative
    elif overall_score == 3:
        return 0  # Neutral
    elif 4 <= overall_score <= 5:
        return 2  # Positive

# Apply the function to the 'overall' column to create the 'sentiment' column
df['sentiment'] = df['overall'].apply(classify_sentiment)

In [26]:
print(df.head())

# Count the number of each sentiment
print(df['sentiment'].value_counts())


   overall                                         reviewText  sentiment
0      5.0  This worked really well for what I used it for...          2
1      5.0                   Fast cutting and good adheasive.          2
2      5.0  Worked great for my lapping bench.  I would li...          2
3      4.0                                      As advertised          2
4      5.0  seems like a pretty good value as opposed to b...          2
sentiment
2    68201
0     4442
1     4417
Name: count, dtype: int64


In [ ]:
# dataset is heavily skewed on the positive side of things so might need to use cross-validation to account for it